<a href="https://colab.research.google.com/github/EziamaUgonna/Bayesian_analysis-/blob/master/workDissertation_work.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip show tensorflow

Name: tensorflow
Version: 1.15.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.6/dist-packages
Requires: keras-preprocessing, astor, six, wrapt, opt-einsum, gast, termcolor, numpy, protobuf, grpcio, keras-applications, tensorflow-estimator, google-pasta, absl-py, tensorboard, wheel
Required-by: stable-baselines, magenta, fancyimpute


In [2]:
!pip show tensorflow_probability

Name: tensorflow-probability
Version: 0.7.0
Summary: Probabilistic modeling and statistical inference in TensorFlow
Home-page: http://github.com/tensorflow/probability
Author: Google LLC
Author-email: no-reply@google.com
License: Apache 2.0
Location: /usr/local/lib/python3.6/dist-packages
Requires: decorator, numpy, cloudpickle, six
Required-by: tensorflow-gan, tensor2tensor, magenta, kfac, dm-sonnet


In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals 

In [4]:
# Dependencies
import os 
import warnings 
#from absl import flags 
import matplotlib 
import numpy as np 
import tensorflow as tf 
import tensorflow_probability as tfp
import math
import pandas as pd
tfd = tfp.distributions
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
#from hyperopt import fmin, tpe, hp, STATUS_OK, STATUS_FAIL, Trials
#import python_utils

In [5]:
flags = tf.app.flags
FLAGS = tf.app.flags.FLAGS
flags.DEFINE_float("learning_rate", default = 0.0001, help = "Initial learning rate.")
flags.DEFINE_integer("epochs", default = 700, help = "Number of epochs to train for")
flags.DEFINE_integer("batch_size", default =128, help = "Batch size.")
flags.DEFINE_integer("eval_freq", default = 400, help =" Frequency at which to validate the model.")
flags.DEFINE_float("kernel_posterior_scale_mean", default = -0.9, help = "Initial kernel posterior mean of the scale (log var) for q(w)")
flags.DEFINE_float("kernel_posterior_scale_constraint", default = 0.2, help = "Posterior kernel constraint for the scale (log var) for q(w)")
flags.DEFINE_float("kl_annealing", default = 50, help = "Epochs to anneal the KL term (anneals from 0 to 1)")
flags.DEFINE_integer("num_hidden_layers", default = 4, help = "Number of hidden layers")
flags.DEFINE_integer("num_monte_carlo",

                     default=50,

                     help="Network draws to compute predictive probabilities.")
tf.app.flags.DEFINE_string('f', '', 'kernel')
#initialize flags 
#FLAGS = flags.FLAGS
print(FLAGS.learning_rate)
print(FLAGS.epochs)
print(FLAGS.num_monte_carlo)

0.0001
700
50


In [0]:
def build_input_pipeline(X_train,X_val,X_test,y_train,y_val,y_test, batch_size, valid_size):
  #Build an iterator over training batches 
  training_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
  #Shuffle the dataset (note shuffle argument much larger than training size)
  # and form batches of size batch_size
  training_batches = training_dataset.shuffle(20000, reshuffle_each_iteration =True).repeat().batch(batch_size)
  training_iterator = tf.data.make_one_shot_iterator(training_batches)
  
  #Building iterator over the heldout set with batch_size = heldout_size,
  # i.e., return the entire heldout set as a constant.
  heldout_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val))
  heldout_batches = heldout_dataset.repeat().batch(valid_size)
  heldout_iterator = tf.data.make_one_shot_iterator(heldout_batches)
  
  test_dataset = tf.data.Dataset.from_tensor_slices((X_test,y_test))
  test_dataset = test_dataset.batch(batch_size)
  test_iterator = test_dataset.make_one_shot_iterator()
  
  
  
  
  #Combine these into a feasible iterator that can switch between training 
  # and validation inputs.
  # Here should be minibatch increment be defined 
  handle = tf.placeholder(tf.string, shape = [])
  feedable_iterator = tf.data.Iterator.from_string_handle(handle, training_batches.output_types, training_batches.output_shapes)
  features_final, labels_final = feedable_iterator.get_next()
  
  return features_final, labels_final, handle, training_iterator, heldout_iterator,test_iterator


In [7]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [8]:
!ls "/content/gdrive/My Drive/"

 01685ea0b1cbf3c8e71f60e0c86bd464.jpg
 20151018_164402.jpg
 20180702_052134.jpg
 2d683da3d7a5c7b98eef15f5056bbd41.jpg
 311096b0040456e8bdb94f5d9aabf0aa.jpg
 400118_3456351547075_502332966_a.jpg
 4f93ea67c48ebc72bd5f585ef349bb57.jpg
 57d38f6f8c6cc4d993f3fd5dec581313.jpg
 9e0f6a555af0ccfc820119a380e864bc.jpg
'About Language. Tasks for teachers of English ( PDFDrive.com ).pdf'
'a survey on trust management for interne of things.pdf'
'Audio from Elvin Eziama'
'Audio from Ugonna'
'Blitzstein Hwang Probability.pdf'
'Chinemezu '
'Colab Notebooks'
'Computational Physics - A Practical Introduction to Computational Physics and Scientific Computing ( PDFDrive.com ).pdf'
'c++ primer.pdf'
 Dessert_code.ipynb
'Dessert_codeV1.2 (1).ipynb'
 Dessert_codeV1.2.ipynb
 Dessert_codeV1F.2.ipynb
 Dropbox.exe
 foo.txt
"FORTUNATE'S TESTIMONIAL20001.jpg"
 Franco.ipynb
'Google Drive.1.pdf'
'IJEOMA (2).JPG'
'IJEOMA (3).jpg'
'IJEOMA (5).JPG'
 IJEOMA.JPG
 IMG_20160828_011517.jpg
 IMG-20170520-WA000.jpg
 IMG-20180528

In [9]:
# Read in the dataset
df = pd.read_csv('/content/gdrive/My Drive/work2.csv')
change = df.query('Speed>0').sample(frac = .1).index
df.loc[change, 'Speed'] = 0
df.loc[change, 'Class'] = 0
df.to_csv('work2.csv', header = True, index =False)
df.shape

(1048575, 20)

In [0]:
X = df.iloc[:,:-1].values.astype(np.float32)
y = df.iloc[:,-1].values.astype(np.int64)

In [11]:
y.dtype

dtype('int64')

In [0]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=200)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=1)


In [0]:
# Standardize the dataset  
scalar_x_train = StandardScaler().fit(X_train)
X_train = scalar_x_train.transform(X_train)
X_test  = scalar_x_train.transform(X_test)
X_val   = scalar_x_train.transform(X_val)

In [14]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 13192611473223073644, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 16991940258924083465
 physical_device_desc: "device: XLA_CPU device"]

In [0]:
#correct_prediction = tf.equal(prediction, tf.argmax(y_labels,1))
#accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))  

In [0]:
def main(argv):
  # extract the activation function from the hyperopt spec as an attribute from the tf.nn module 
  #activation = getattr(tf.nn, FLAGS.activation_function)
  # define the graph 
  #with tf.Graph().as_default():
  (features_final, labels_final, handle, training_iterator, heldout_iterator,test_iterator) = build_input_pipeline(X_train,X_val,X_test,y_train,y_val,y_test, FLAGS.batch_size, 500)
  
  
  # Building the Bayesian Neural Network 
  # we are Gaussian Reparametrization Trick 
  # to compute the stochastic gradients as described in the paper 
  with tf.compat.v1.name_scope("bayesian_neural_net", values =[features_final]):
    neural_net = tf.keras.Sequential()
    for i in range(FLAGS.num_hidden_layers):
      layer = tfp.layers.DenseReparameterization(
          units = 10,
          activation = tf.nn.relu,
          trainable = True,
          kernel_prior_fn=tfp.layers.default_multivariate_normal_fn, # NormalDiag
          kernel_posterior_fn=tfp.layers.default_mean_field_normal_fn(),
          #kernel_posterior_fn=tfp_layers_util.default_mean_field_normal_fn(), # softplus(sigma)
          kernel_posterior_tensor_fn=lambda x: x.sample(),
          bias_prior_fn=tfp.layers.default_multivariate_normal_fn, # NormalDiag
          bias_posterior_fn=tfp.layers.default_mean_field_normal_fn(), # softplus(sigma)
          bias_posterior_tensor_fn=lambda x: x.sample()
          )
      neural_net.add(layer)
  neural_net.add(tfp.layers.DenseReparameterization(
      units=2, # one dimensional output
      activation= tf.nn.softmax, # since regression (outcome not bounded)
      trainable=True, # i.e subject to optimization
      kernel_prior_fn=tfp.layers.default_multivariate_normal_fn, # NormalDiag with hyperopt sigma
      kernel_posterior_fn=tfp.layers.default_mean_field_normal_fn(), # softplus(sigma)
      kernel_posterior_tensor_fn=lambda x: x.sample(),
      bias_prior_fn =tfp.layers.default_multivariate_normal_fn, # NormalDiag with hyperopt sigma
      bias_posterior_fn=tfp.layers.default_mean_field_normal_fn(), # softplus(sigma)
      bias_posterior_tensor_fn=lambda x: x.sample()
      ))
  logits = neural_net(features_final)
  #labels_distribution = tfd.Bernoulli(logits=logits)
  labels_distribution = tfd.Categorical(logits=logits)
  #labels_distribution = tfd.Bernoulli(logits=logits)
  
  # Perform KL annealing. The optimal number of annealing steps 
  # depends on the dataset and architecture.
  t = tf.Variable(0.0)
  kl_regularizer = t / (FLAGS.kl_annealing * len(X_train) / FLAGS.batch_size)
  
  #Compute the -ELBO as the loss. The kl term is annealed from 1 to 1 over 
  # the epochs specified by the kl_annealing flag.
  log_likelihood = labels_distribution.log_prob(labels_final)
  #neg_log_likelihood = tf.reduce_mean(tf.squared_difference(logits,labels_final))
  neg_log_likelihood = -tf.reduce_mean(input_tensor = log_likelihood)
  kl = sum(neural_net.losses)/len(X_train) * tf.minimum(1.0, kl_regularizer)
  elbo_loss = neg_log_likelihood + kl
  
  # Build metrics for evaluation. Predictions are formed from single forward 
  # pass of the probablisitic layers . They are cheap but noisy predictions
  predictions = tf.argmax(input = logits, axis=1)
  #predictions = tf.cast(predictions, tf.int32)
  # TP, TN, FP, FN
  TP = tf.count_nonzero(predictions * labels_final)
  TN = tf.count_nonzero((predictions - 1) * (labels_final - 1))
  FP = tf.count_nonzero(predictions * (labels_final - 1))
  FN = tf.count_nonzero((predictions - 1) * labels_final)
  # precision, recall, f1
  precision = TP / (TP + FP)
  recall = TP / (TP + FN)
  f1 = 2 * precision * recall / (precision + recall)
  
  tpr = TP/(TP+FN)
  fpr = FP/(TP+FN)
  with tf.compat.v1.name_scope("train"):
    correct_prediction = tf.equal(predictions, tf.argmax(labels_final))
    training_accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
    
    
    #train_accuracy, train_accuracy_update_op = tf.metrics.accuracy(labels=labels_final,predictions =predictions)
    opt = tf.train.AdamOptimizer(FLAGS.learning_rate)
    train_op = opt.minimize(elbo_loss)
    update_step_op = tf.assign(t, t+1)
  
  with tf.compat.v1.name_scope("valid"):
    correct_prediction_val = tf.equal(predictions, tf.argmax(labels_final))
    valid_accuracy = tf.reduce_mean(tf.cast(correct_prediction_val, tf.float32))
    
    #valid_accuracy, validation_accuracy_update_op = tf.metrics.accuracy(labels= labels_final,predictions = predictions)
  with tf.compat.v1.name_scope("test"):
    
    correct_prediction_test = tf.equal(predictions, tf.argmax(labels_final))
    test_accuracy = tf.reduce_mean(tf.cast(correct_prediction_test, tf.float32))
    test_accuracy, test_accuracy_update_op = tf.metrics.accuracy(labels = labels_final,predictions = predictions)
  
  init_op = tf.group(tf.global_variables_initializer(),
                     tf.local_variables_initializer())
  saver = tf.train.Saver()
  
 # stream_vars_valid = [ v for v in tf.local_variables() if "valid" in v.name]
  
  #reset_valid_op = tf.variables_initializer(stream_vars_valid)
  
  valid_accuracy_summary = []
  stop_early =0
  with tf.compat.v1.Session() as sess:
    sess.run(init_op)
    
    # Run the training loop

    train_handle = sess.run(training_iterator.string_handle())
    heldout_handle = sess.run(  heldout_iterator.string_handle())
      
    test_handle = sess.run(test_iterator.string_handle())

    training_steps = int(round(FLAGS.epochs * (len(X_train) / FLAGS.batch_size)))

    for step in range(training_steps):
      
      _ = sess.run([train_op],feed_dict={handle: train_handle})

     # _ = sess.run([train_op,train_accuracy_update_op, update_step_op],feed_dict={handle: train_handle})
      
      # Manually print the frequency 
      if step % 100 == 0:
        save_path = saver.save(sess, "/tmp/my_model.ckpt")
        loss_value, accuracy_value, kl_value = sess.run([elbo_loss, training_accuracy, kl], feed_dict= {handle:train_handle})
        print("Step:{:>3d} loss : {:.3f} KL: {:.3f}" .format(step , loss_value, accuracy_value, kl_value))
        
        
      if (step +1) % FLAGS.eval_freq ==0: 
        # Compute log prob of heldout set by averaging draws from the model:
        # p(heldout | train) = int_model p(heldout|model) p(model|train) ~= 1/n * sum_{i=1}^n p(heldout | model_i)
        # where model_i is a draw from the posterior 
        #p(model|train)
        probs = np.asarray([sess.run((labels_distribution.probs), 
                                     feed_dict ={handle: heldout_handle})
      
                            for _ in range(FLAGS.num_monte_carlo)])
        mean_probs = np.mean(probs, axis =0).astype(np.int32)
        print(mean_probs.dtype)
        
        _, label_vals = sess.run((features_final, labels_final), feed_dict = {handle: heldout_handle})
        #label_vals = (label_vals).astype(np.int32)
      
        heldout_lp = np.mean(np.log(mean_probs[np.arange(mean_probs.shape[0]), label_vals]))
        
        
        print(" ...Held_out nats: {:.3f}".format(heldout_lp))
        
       # Calculate validation accuracy

        for step in range(10):

          #sess.run(validation_accuracy_update_op, feed_dict={handle: heldout_handle})

          valid_value = sess.run(valid_accuracy, feed_dict={handle: heldout_handle})
        
          valid_accuracy_summary.append(valid_value)
          if valid_value < max(valid_accuracy_summary) and step > 100:
            stop_early += 1
          if stop_early == 15:
            break
          else:
            stop_early = 0
        print("Validation Accuracy: {:.3f}".format(valid_value))
        #sess.run(reset_valid_op)
        
        #sess.run(test_accuracy_update_op, feed_dict ={handle:test_handle})
        #test_value = sess.run(test_accuracy, feed_dict= {handle:test_handle})
          
        test_value, _ = sess.run([test_accuracy, test_accuracy_update_op], feed_dict={handle: test_handle})
        
        loss_value,test_value, precision_value, recall_value, fpr_value, tpr_value,f1_value = sess.run([elbo_loss,test_accuracy, precision, recall, fpr, tpr,f1],feed_dict={handle: test_handle})
        
        print("Step: {:>3d} Loss: {:.3f}  test Accuracy: {:.3f} Precision: {:.3f} Recall: {:.3f} ".format(step, loss_value, test_value, precision_value, recall_value))
        
        print("Step: {:>3d} fpr: {:.3f} tpr: {:.3f} f1_1: {:.3f}".format( step, fpr_value, tpr_value,f1_value))
        
        
        
        
if __name__ == "__main__":
  tf.compat.v1.app.run()
  